# Detecting Youtube Comment Spam

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Soohwan-Lee/hcmld21/main/Datasets/Youtube01-Psy.csv"
d = pd.read_csv(url)

In [2]:
len(d.query('CLASS == 1'))

175

In [3]:
len(d.query('CLASS == 0'))

175

In [4]:
len(d)

350

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [6]:
dvec = vectorizer.fit_transform(d['CONTENT'])

In [7]:
dvec

<350x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 4354 stored elements in Compressed Sparse Row format>

In [8]:
vectorizer.get_feature_names()

['00',
 '000',
 '02',
 '034',
 '05',
 '08',
 '10',
 '100',
 '100000415527985',
 '10200253113705769',
 '1030',
 '1073741828',
 '11',
 '1111',
 '112720997191206369631',
 '12',
 '123',
 '124',
 '124923004',
 '126',
 '127',
 '13017194',
 '131338190916',
 '1340488',
 '1340489',
 '1340490',
 '1340491',
 '1340492',
 '1340493',
 '1340494',
 '1340499',
 '1340500',
 '1340502',
 '1340503',
 '1340504',
 '1340517',
 '1340518',
 '1340519',
 '1340520',
 '1340521',
 '1340522',
 '1340523',
 '1340524',
 '134470083389909',
 '1415297812',
 '1495323920744243',
 '1496241863981208',
 '1496273723978022',
 '1498561870415874',
 '161620527267482',
 '171183229277',
 '19',
 '19924',
 '1firo',
 '1m',
 '20',
 '2009',
 '2012',
 '2012bitches',
 '2013',
 '2014',
 '201470069872822',
 '2015',
 '2017',
 '210',
 '23',
 '24',
 '24398',
 '243a',
 '279',
 '29',
 '2b',
 '2billion',
 '2x10',
 '300',
 '3000',
 '313327',
 '315',
 '322',
 '33',
 '33gxrf',
 '39',
 '390875584405933',
 '391725794320912',
 '40beuutvu2zkxk4utgpz8k',
 '

In [9]:
dshuf = d.sample(frac=1)

In [10]:
d_train = dshuf[:300]
d_test = dshuf[300:]
d_train_att = vectorizer.transform(d_train['CONTENT'])
d_test_att = vectorizer.transform(d_test['CONTENT'])
d_train_label = d_train['CLASS']
d_test_label = d_test['CLASS']

In [11]:
d_train_att

<300x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 3650 stored elements in Compressed Sparse Row format>

In [12]:
d_test_att

<50x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 704 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)

In [14]:
clf.fit(d_train_att,d_train_label)

RandomForestClassifier(n_estimators=80)

In [15]:
clf.score(d_test_att,d_test_label)

0.94

In [16]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)

array([[26,  1],
       [ 2, 21]], dtype=int64)

In [17]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()*2))

Accuracy: 0.96 (+/- 0.02)


In [18]:
firstUrl = "https://raw.githubusercontent.com/Soohwan-Lee/hcmld21/main/Datasets/Youtube01-Psy.csv"
secondUrl = "https://raw.githubusercontent.com/Soohwan-Lee/hcmld21/main/Datasets/Youtube02-KatyPerry.csv"
thridUrl = "https://raw.githubusercontent.com/Soohwan-Lee/hcmld21/main/Datasets/Youtube03-LMFAO.csv"
fourthUrl = "https://raw.githubusercontent.com/Soohwan-Lee/hcmld21/main/Datasets/Youtube04-Eminem.csv"
fifthUrl = "https://raw.githubusercontent.com/Soohwan-Lee/hcmld21/main/Datasets/Youtube05-Shakira.csv"

d = pd.concat([pd.read_csv(firstUrl),
              pd.read_csv(secondUrl),
              pd.read_csv(thridUrl),
              pd.read_csv(fourthUrl),
              pd.read_csv(fifthUrl)])

In [19]:
len(d)

1956

In [20]:
len(d.query('CLASS == 1'))

1005

In [21]:
len(d.query('CLASS == 0'))

951

In [22]:
dshuf = d.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']

In [23]:
from sklearn.pipeline import Pipeline, make_pipeline
pipeline = Pipeline([
    ('bag-of-words', CountVectorizer()),
    ('random forest', RandomForestClassifier()),
])
pipeline

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [24]:
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [25]:
pipeline.fit(d_content[:1500],d_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [26]:
pipeline.score(d_content[1500:],d_label[1500:])

0.9671052631578947

In [27]:
pipeline.predict(["what a neat video!"])

array([0], dtype=int64)

In [28]:
pipeline.predict(["plz subscribe to my channel"])

array([1], dtype=int64)

In [29]:
scores = cross_val_score(pipeline, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()*2))

Accuracy: 0.96 (+/- 0.01)


In [30]:
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(), TfidfTransformer(norm=None),RandomForestClassifier())

In [31]:
scores = cross_val_score(pipeline2, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() *2))

Accuracy: 0.96 (+/- 0.01)


In [32]:
pipeline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]

In [33]:
parameters = {
    'countvectorizer__max_features': (None, 1000, 2000),
    'countvectorizer__ngram_range': ((1, 1), (1, 2)), 
    'countvectorizer__stop_words': ('english', None),
    'tfidftransformer__use_idf': (True, False), 
    'randomforestclassifier__n_estimators': (20, 50, 100)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=-1, verbose=1)

In [34]:
grid_search.fit(d_content, d_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   12.0s finished


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

In [35]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.961
Best parameters set:
	countvectorizer__max_features: 2000
	countvectorizer__ngram_range: (1, 1)
	countvectorizer__stop_words: 'english'
	randomforestclassifier__n_estimators: 20
	tfidftransformer__use_idf: True


# 4.2 Test any sentance whether it is spam or not
For most comment, it works well, however, it sometimes does not work well for some tricky comments. The last comment was what I wrote to induce people to search my channel. The bag of words model have not data for similarity of words. Also, if there are new type of spam comment style out of existing data set, I think it is hard to figure out whether it is spam comment or not. 

In [40]:
pipeline.predict(["I think they like me"])

array([0], dtype=int64)

In [41]:
pipeline.predict(["are your dislikes prepared for rewind 2019?"])

array([0], dtype=int64)

In [42]:
pipeline.predict(["You should check my channel for Funny VIDEOS!!"])

array([1], dtype=int64)

In [43]:
pipeline.predict(["just for test I have to say murdev.com"])

array([1], dtype=int64)

In [45]:
pipeline.predict(["It is time to search 'SWAN STUDIO'!!!"])

array([0], dtype=int64)

# 4.3 Using trigrams in n-gram-range

In [50]:
parameters = {
    'countvectorizer__max_features': (None, 1000, 2000),
    'countvectorizer__ngram_range': ((1,3),(1, 3)), # using trigrams
    'countvectorizer__stop_words': ('english', None),
    'tfidftransformer__use_idf': (True, False), 
    'randomforestclassifier__n_estimators': (20, 50, 100)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=-1, verbose=1)

In [51]:
grid_search.fit(d_content, d_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   12.9s finished


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 3), (1, 3)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

In [52]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.963
Best parameters set:
	countvectorizer__max_features: 2000
	countvectorizer__ngram_range: (1, 3)
	countvectorizer__stop_words: 'english'
	randomforestclassifier__n_estimators: 50
	tfidftransformer__use_idf: True


# 4.4 Apply the parameters to build a new model
I used the unigram or bigrams as shown in the original code, not trigrams on exercise 4.3.

In [64]:
best_model = grid_search.best_estimator_

In [67]:
scores = cross_val_score(best_model, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() *2))

Accuracy: 0.96 (+/- 0.01)


# 4.5 Complete the Word2Vec

In [59]:
import gensim,logging

In [60]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

In [64]:
gmodel = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',binary=True)

2021-03-31 10:31:12,825 : INFO : loading projection weights from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
2021-03-31 10:34:59,742 : INFO : loaded (3000000, 300) matrix from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz


In [65]:
gmodel['cat']

array([ 0.0123291 ,  0.20410156, -0.28515625,  0.21679688,  0.11816406,
        0.08300781,  0.04980469, -0.00952148,  0.22070312, -0.12597656,
        0.08056641, -0.5859375 , -0.00445557, -0.296875  , -0.01312256,
       -0.08349609,  0.05053711,  0.15136719, -0.44921875, -0.0135498 ,
        0.21484375, -0.14746094,  0.22460938, -0.125     , -0.09716797,
        0.24902344, -0.2890625 ,  0.36523438,  0.41210938, -0.0859375 ,
       -0.07861328, -0.19726562, -0.09082031, -0.14160156, -0.10253906,
        0.13085938, -0.00346375,  0.07226562,  0.04418945,  0.34570312,
        0.07470703, -0.11230469,  0.06738281,  0.11230469,  0.01977539,
       -0.12353516,  0.20996094, -0.07226562, -0.02783203,  0.05541992,
       -0.33398438,  0.08544922,  0.34375   ,  0.13964844,  0.04931641,
       -0.13476562,  0.16308594, -0.37304688,  0.39648438,  0.10693359,
        0.22167969,  0.21289062, -0.08984375,  0.20703125,  0.08935547,
       -0.08251953,  0.05957031,  0.10205078, -0.19238281, -0.09

In [66]:
gmodel.similarity('cat','dog')

0.76094574

In [68]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [80]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent)
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent)
    sent = re.sub(r'\W', ' ', sent) 
    sent = re.sub(r'\s+', ' ', sent) 
    sent = sent.strip()
    return sent.split()

In [81]:
import re
import os
unsup_sentences = []

for dirname in ["train/pos", "train/neg", "train/unsup", "test/pos", "test/neg"]:
    for fname in sorted(os.listdir("aclImdb/" + dirname)):
        if fname[-4:] == '.txt':
            with open("aclImdb/" + dirname + "/" + fname, encoding='UTF-8') as f:
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname + "/" + fname]))
                
for dirname in ["txt_sentoken/pos", "txt_sentoken/neg"]:
    for fname in sorted(os.listdir(dirname)):
        if fname[-4:] == '.txt':
            with open(dirname + "/" + fname, encoding='UTF-8') as f:
                for i, sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words, ["%s/%s-%d" % (dirname, fname, i)]))
                    
with open("stanfordSentimentTreebank/original_rt_snippets.txt", encoding='UTF-8') as f:
    for i, line in enumerate(f):
        words = extract_words(sent)
        unsup_sentences.append(TaggedDocument(words, ["rt-%d" % i]))


In [82]:
len(unsup_sentences)

175325

In [83]:
unsup_sentences[0:10]

[TaggedDocument(words=['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such', 'as', 'teachers', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'hig', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', 'the', 'scramble', 'to', 'survive', 'financially', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', 'i', 'immediately', 'recalled', 'at', 'high', 'a', 'classic', 'line', 'inspector', 'here', 'to', 'sack', 'one', 'of', 'your', 'teachers', 'student', 'welcome', 

In [104]:
### REF: https://github.com/PacktPublishing/Python-Artificial-Intelligence-Projects-for-Beginners/blob/master/Chapter03/SentimentAnalysis.py
# I needed to use PermuteSentence Class, thus I googled as following keywords "python PermuteSentence Class"
import random
class PermuteSentences(object):
    def __init__(self, sents):
        self.sents = sents
        
    def __iter__(self):
        shuffled = list(self.sents)
        random.shuffle(shuffled)
        for sent in shuffled:
            yield sent

In [105]:
permuter = PermuteSentences(unsup_sentences) 
model = Doc2Vec(permuter, dm=0, hs=1, size=50)

C:\Users\LeeSooHwan\anaconda3\envs\py37\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2021-03-31 11:19:03,730 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-03-31 11:19:03,731 : INFO : collecting all words and their counts
2021-03-31 11:19:03,843 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-03-31 11:19:04,038 : INFO : PROGRESS: at example #10000, processed 1397256 words (7152515/s), 45097 word types, 10000 tags
2021-03-31 11:19:04,226 : INFO : PROGRESS: at example #20000, processed 2812774 words (7588757/s), 61407 word types, 20000 tags
2021-03-31 11:19:04,416 : INFO : PROGRESS: at example #30000, processed 4217786 words (7436404/s), 73197 word types, 30000 tags
2021-03-31 11:19:04,607 : INFO : PRO

2021-03-31 11:20:16,245 : INFO : EPOCH 2 - PROGRESS: at 89.92% examples, 1046764 words/s, in_qsize 5, out_qsize 0
2021-03-31 11:20:17,265 : INFO : EPOCH 2 - PROGRESS: at 95.68% examples, 1049010 words/s, in_qsize 5, out_qsize 0
2021-03-31 11:20:17,992 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-31 11:20:17,994 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-31 11:20:18,007 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-31 11:20:18,008 : INFO : EPOCH - 2 : training on 24693510 raw words (18753288 effective words) took 17.8s, 1050855 effective words/s
2021-03-31 11:20:19,019 : INFO : EPOCH 3 - PROGRESS: at 5.08% examples, 937129 words/s, in_qsize 5, out_qsize 0
2021-03-31 11:20:20,022 : INFO : EPOCH 3 - PROGRESS: at 10.71% examples, 989367 words/s, in_qsize 5, out_qsize 0
2021-03-31 11:20:21,025 : INFO : EPOCH 3 - PROGRESS: at 16.42% examples, 1016740 words/s, in_qsize 5, out_qsize 0
2021-03-31 11:20:22

In [106]:
model.delete_temporary_training_data(keep_inference=True)

In [107]:
model.save('review.d2v')

2021-03-31 11:21:10,021 : INFO : saving Doc2Vec object under review.d2v, separately None
2021-03-31 11:21:11,590 : INFO : saved review.d2v


In [108]:
model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))

array([-0.2354145 , -0.0747091 ,  0.23725848, -0.16804855,  0.03809171,
       -0.26735035,  0.10125965,  0.03975399, -0.0538856 ,  0.15884551,
       -0.10138144, -0.346097  ,  0.38475174,  0.06646588, -0.296666  ,
        0.02790331,  0.12379564, -0.10781914,  0.129481  , -0.08635166,
        0.14061613, -0.10235769, -0.23111059,  0.07314836,  0.22970548,
       -0.02419515, -0.12309412,  0.40725428, -0.04017804, -0.08313112,
       -0.35634443,  0.01005906, -0.0443926 , -0.06115242,  0.41289884,
       -0.01377894,  0.16390868, -0.16181411,  0.14085692,  0.03785241,
        0.09464942, -0.36501098,  0.4251935 ,  0.08761124, -0.2813565 ,
       -0.08492973,  0.0023178 , -0.18295443, -0.02106301,  0.04326778],
      dtype=float32)

In [109]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
    [model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))],
    [model.infer_vector(extract_words("Service sucks"))]
)

array([[0.50089896]], dtype=float32)

In [110]:
cosine_similarity(
    [model.infer_vector(extract_words("Highly recommended"))],
    [model.infer_vector(extract_words("Service sucks"))]
)

array([[0.26085353]], dtype=float32)

In [111]:
sentences = []
sentvecs = []
sentiments = []
for fname in ["yelp", "amazon_cells", "imdb"]: 
    with open("sentiment labelled sentences/%s_labelled.txt" % fname, encoding='UTF-8') as f:
        for i, line in enumerate(f):
            line_split = line.strip().split('\t')
            sentences.append(line_split[0])
            words = extract_words(line_split[0])
            sentvecs.append(model.infer_vector(words, steps=10))
            sentiments.append(int(line_split[1]))
            
combined = list(zip(sentences, sentvecs, sentiments))
random.shuffle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [112]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

In [113]:
scores = cross_val_score(clf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.7866666666666666, 0.013291601358251255)

In [114]:
scores = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.782, 0.013182479955523478)

In [115]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())

In [116]:
scores = cross_val_score(pipeline, sentences, sentiments, cv=5)
np.mean(scores), np.std(scores)

(0.7756666666666666, 0.009043106644167017)